In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras

In [2]:
df = pd.read_csv('Data Sets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 1)

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [7]:
model = Sequential()

model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

C:\Users\Anindya Dolui\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4359 - loss: 0.7401 - val_accuracy: 0.6364 - val_loss: 0.6483
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6101 - loss: 0.6840 - val_accuracy: 0.7273 - val_loss: 0.6079
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7026 - loss: 0.6299 - val_accuracy: 0.7857 - val_loss: 0.5745
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7513 - loss: 0.5975 - val_accuracy: 0.7857 - val_loss: 0.5495
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7494 - loss: 0.5870 - val_accuracy: 0.7792 - val_loss: 0.5297
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7596 - loss: 0.5399 - val_accuracy: 0.7857 - val_loss: 0.5141
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7668 - loss: 0.5298 - val_accuracy: 0.7857 - val_loss: 0.5031
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7844 - loss: 0.4998 - val_accuracy: 0.7727 - 

Combining all the steps that were implemented individually

In [9]:
import keras_tuner as kt

In [10]:
def build_model(hp):
    model = Sequential()

    counter = 0
    for i in range(hp.Int('num_layers', min_value = 1, max_value = 10)):

        if counter == 0:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                    activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid']),
                    input_dim = 8
                    )
                )
        else:
            model.add(
                Dense(
                    hp.Int('units' + str(i), min_value = 8, max_value = 128, step = 8),
                    activation = hp.Choice('activation' + str(i), values = ['relu', 'tanh', 'sigmoid'])
                    )
                )
        counter += 1
    
    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer = hp.Choice('optimizer', values = ['adam', 'rmsprop', 'sgd', 'nadam', 'adadelta']),
                 loss = 'binary_crossentropy',
                 metrics = ['accuracy'])
    return model
            

In [11]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'Hyperparameter Tuning json',
    project_name = 'All in One Model using Keras Tuner'
)

In [12]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 06s]
val_accuracy: 0.8181818127632141

Best val_accuracy So Far: 0.8181818127632141
Total elapsed time: 00h 00m 23s


In [17]:
tuner.get_best_hyperparameters()[0].values

# total no of hidden layers -> 4
# first hidden layer -> no no nodes = 56, activation = relu, optimizer = nadam
# like this...all the accurate infos are automated for all the layers using keras tuner

{'num_layers': 4,
 'units0': 56,
 'activation0': 'relu',
 'optimizer': 'nadam',
 'units1': 80,
 'activation1': 'tanh',
 'units2': 64,
 'activation2': 'sigmoid',
 'units3': 24,
 'activation3': 'tanh',
 'units4': 120,
 'activation4': 'tanh',
 'units5': 56,
 'activation5': 'relu',
 'units6': 80,
 'activation6': 'sigmoid',
 'units7': 56,
 'activation7': 'relu'}

In [14]:
model = tuner.get_best_models(num_models = 1)[0]

C:\Users\Anindya Dolui\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 23 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 56)             │           504 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 80)             │         4,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         5,184 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 24)             │         1,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,833 (46.22 KB)

 Trainable params: 11,833 (46.22 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 15ms/step - accuracy: 0.7783 - loss: 0.4720 - val_accuracy: 0.7857 - val_loss: 0.4702
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7545 - loss: 0.4755 - val_accuracy: 0.7922 - val_loss: 0.4634
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7809 - loss: 0.4554 - val_accuracy: 0.7987 - val_loss: 0.4617
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7658 - loss: 0.4593 - val_accuracy: 0.8052 - val_loss: 0.4584
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7910 - loss: 0.4516 - val_accuracy: 0.7727 - val_loss: 0.4603
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7570 - loss: 0.4617 - val_accuracy: 0.7987 - val_loss: 0.4584
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8132 - loss: 0.4280 - val_accuracy: 0.7987 - val_loss: 0.4593
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7829 - loss: 0.4249 - val_accuracy: 0.78

previous accuracy was 78% where everything was manually defined
but after tuning the hyperparamters, the accuracy is around 90%